In [9]:
from ultralytics import YOLO
import cv2
import time
import os
import pandas as pd

model = YOLO("yolov8n.pt")

image_folder = "images"
valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp")

results_data = []

for img_name in os.listdir(image_folder):
    if not img_name.lower().endswith(valid_ext):
        continue   # ✅ skips .DS_Store and any non-image files

    img_path = os.path.join(image_folder, img_name)

    start = time.time()
    results = model(img_path)
    end = time.time()

    detections = results[0].boxes
    objects = len(detections)
    avg_conf = float(detections.conf.mean()) if objects > 0 else 0

    results_data.append({
        "Model": "YOLOv8",
        "Image": img_name,
        "Objects Detected": objects,
        "Avg Confidence": round(avg_conf, 3),
        "Time (sec)": round(end - start, 4)
    })

df_yolo = pd.DataFrame(results_data)
df_yolo


image 1/1 /Users/klaus/Desktop/MSBA - 503/images/clark_bball.webp: 384x640 8 persons, 96.0ms
Speed: 2.7ms preprocess, 96.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/klaus/Desktop/MSBA - 503/images/dupont.jpg: 352x640 11 persons, 2 sports balls, 101.9ms
Speed: 1.8ms preprocess, 101.9ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)

image 1/1 /Users/klaus/Desktop/MSBA - 503/images/kai.webp: 448x640 2 persons, 1 bottle, 96.2ms
Speed: 4.3ms preprocess, 96.2ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/klaus/Desktop/MSBA - 503/images/big_ears.webp: 448x640 1 cat, 69.9ms
Speed: 2.3ms preprocess, 69.9ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/klaus/Desktop/MSBA - 503/images/united.jpg: 448x640 1 airplane, 89.2ms
Speed: 3.1ms preprocess, 89.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/klaus/Desktop/MSBA - 503

,Model,Image,Objects Detected,Avg Confidence,Time (sec)
0,YOLOv8,clark_bball.webp,8,0.625,0.1417
1,YOLOv8,dupont.jpg,13,0.559,0.1126
2,YOLOv8,kai.webp,3,0.773,0.1132
3,YOLOv8,big_ears.webp,1,0.656,0.1349
4,YOLOv8,united.jpg,1,0.923,0.1050
5,YOLOv8,sports_fan.jpeg,4,0.542,0.1001
6,YOLOv8,chair.jpg,3,0.796,0.1605
7,YOLOv8,fruit.jpg,2,0.512,0.2485
8,YOLOv8,sandiego.jpg,18,0.540,0.2407
9,YOLOv8,dogs.webp,9,0.850,0.1078


In [12]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

# Load pre-trained Faster R-CNN
frcnn_model = fasterrcnn_resnet50_fpn(pretrained=True)
frcnn_model.eval()

frcnn_results_data = []

for img_name in os.listdir(image_folder):
    if not img_name.lower().endswith(valid_ext):
        continue

    img_path = os.path.join(image_folder, img_name)
    img = Image.open(img_path).convert("RGB")
    img_tensor = F.to_tensor(img)

    start = time.time()
    with torch.no_grad():
        outputs = frcnn_model([img_tensor])
    end = time.time()

    boxes = outputs[0]['boxes']
    scores = outputs[0]['scores']
    objects = len(boxes)
    avg_conf = float(scores.mean()) if objects > 0 else 0

    frcnn_results_data.append({
        "Model": "Faster-RCNN",
        "Image": img_name,
        "Objects Detected": objects,
        "Avg Confidence": round(avg_conf, 3),
        "Time (sec)": round(end - start, 4)
    })

df_frcnn = pd.DataFrame(frcnn_results_data)

/opt/anaconda3/envs/tf/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/tf/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
df_comparison = pd.concat([df_yolo, df_frcnn], ignore_index=True)
print(df_comparison)

          Model             Image  Objects Detected  Avg Confidence  \
0        YOLOv8  clark_bball.webp                 8           0.625   
1        YOLOv8        dupont.jpg                13           0.559   
2        YOLOv8          kai.webp                 3           0.773   
3        YOLOv8     big_ears.webp                 1           0.656   
4        YOLOv8        united.jpg                 1           0.923   
5        YOLOv8   sports_fan.jpeg                 4           0.542   
6        YOLOv8         chair.jpg                 3           0.796   
7        YOLOv8         fruit.jpg                 2           0.512   
8        YOLOv8      sandiego.jpg                18           0.540   
9        YOLOv8         dogs.webp                 9           0.850   
10  Faster-RCNN  clark_bball.webp               100           0.431   
11  Faster-RCNN        dupont.jpg                55           0.355   
12  Faster-RCNN          kai.webp                29           0.284   
13  Fa

In [14]:
additional_data = []

for img_name in os.listdir(image_folder):
    if not img_name.lower().endswith(valid_ext):
        continue

    img_path = os.path.join(image_folder, img_name)
    img = cv2.imread(img_path)
    h, w, c = img.shape
    avg_color = img.mean(axis=(0,1))

    additional_data.append({
        "Image": img_name,
        "Width": w,
        "Height": h,
        "Avg Color (BGR)": avg_color.round(2)
    })

df_additional = pd.DataFrame(additional_data)
print(df_additional)


              Image  Width  Height           Avg Color (BGR)
0  clark_bball.webp    640     360     [72.92, 67.84, 73.06]
1        dupont.jpg   1050     550    [75.64, 118.36, 99.67]
2          kai.webp    942     628     [41.81, 47.04, 55.65]
3     big_ears.webp   2500    1667   [86.99, 124.24, 178.22]
4        united.jpg   1024     683  [171.87, 142.31, 115.57]
5   sports_fan.jpeg   1000     562     [86.84, 96.2, 142.55]
6         chair.jpg   1500    1500  [131.59, 153.43, 172.55]
7         fruit.jpg   4000    2667     [59.4, 89.09, 106.08]
8      sandiego.jpg   2560    1965   [72.78, 101.59, 147.77]
9         dogs.webp    520     291  [135.13, 150.53, 168.51]
